In [1]:
import time
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import matplotlib.image as mpimg
from collections import OrderedDict
from skimage import io, transform
from math import *
import xml.etree.ElementTree as ET 

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
class Transforms():
    def __init__(self):
        pass
    
    def rotate(self, image, landmarks, angle):
        angle = random.uniform(-angle, +angle)

        transformation_matrix = torch.tensor([
            [+cos(radians(angle)), -sin(radians(angle))], 
            [+sin(radians(angle)), +cos(radians(angle))]
        ])

        image = imutils.rotate(np.array(image), angle)

        landmarks = landmarks - 112
        new_landmarks = np.matmul(landmarks, transformation_matrix)
        new_landmarks = new_landmarks + 112
        return Image.fromarray(image), new_landmarks

    def resize(self, image, landmarks, img_size):
        ###
        landmarks = landmarks / torch.tensor([np.array(image).shape[0]/224, np.array(image).shape[0]/224])
        ###
        image = TF.resize(image, img_size)
        return image, landmarks

    def color_jitter(self, image, landmarks):
        color_jitter = transforms.ColorJitter(brightness=0.3, 
                                              contrast=0.3,
                                              saturation=0.3, 
                                              hue=0.1)
        image = color_jitter(image)
        return image, landmarks

    def crop_face(self, image, landmarks, crops):
        left = int(crops['left'])
        top = int(crops['top'])
        width = int(crops['width'])
        height = int(crops['height'])
        
        if(width > height) :
            left = left - width*0.25
            top = top + height*0.5 - width*0.5 - width*0.25
            width = width*1.5
            height = width
        else :
            top = top - height*0.25
            left = left + width*0.5 - height*0.5 - height*0.25
            height = height*1.5
            width = height

        image = TF.crop(image, top, left, height, width)
        
        img_shape = np.array(image).shape
        landmarks = torch.tensor(landmarks) - torch.tensor([[left, top]])
        #landmarks = landmarks / torch.tensor([img_shape[1], img_shape[0]])
        return image, landmarks

    def __call__(self, image, landmarks, crops):
        image = Image.fromarray(image)
        image, landmarks = self.crop_face(image, landmarks, crops)
        image, landmarks = self.resize(image, landmarks, (224, 224))
        image, landmarks = self.color_jitter(image, landmarks)
        image, landmarks = self.rotate(image, landmarks, angle=10)
        
        image = TF.to_tensor(image)
        image = TF.normalize(image, [0.5], [0.5])
        
        return image, landmarks

In [3]:
class FaceLandmarksDataset(Dataset):

    def __init__(self, transform=None):
        tree = ET.parse('./300W/labels_300W_new.xml')
        root = tree.getroot()

        self.image_filenames = []
        self.landmarks = []
        self.crops = []
        self.transform = transform
        self.root_dir= './300W'
        
        
        for filename in root[0]:
            # pass only 0s.
            #if filename.attrib['file'].split('.')[-2][-2:] != '_0' :
            #    #print(filename.attrib['file'])
            #    continue
            
            self.image_filenames.append(os.path.join(self.root_dir, filename.attrib['file']))

            self.crops.append(filename[0].attrib)

            landmark = []
            for num in range(68):
                x_coordinate = int(filename[0][num].attrib['x'])
                y_coordinate = int(filename[0][num].attrib['y'])
                landmark.append([x_coordinate, y_coordinate])
            self.landmarks.append(landmark)

        self.landmarks = np.array(self.landmarks).astype('float32')     

        assert len(self.image_filenames) == len(self.landmarks)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        image = cv2.imread(self.image_filenames[index], 0)
        landmarks = self.landmarks[index]
        
        if self.transform:
            image, landmarks = self.transform(image, landmarks, self.crops[index])

        #landmarks = landmarks - 0.5

        return image, landmarks

dataset = FaceLandmarksDataset(Transforms())

In [4]:
# split the dataset into validation and test sets
len_valid_set = int(0.1*len(dataset))
len_train_set = len(dataset) - len_valid_set

print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_set))

train_dataset , valid_dataset,  = torch.utils.data.random_split(dataset , [len_train_set, len_valid_set])

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=2, shuffle=True, num_workers=0)

The length of Train set is 3426
The length of Valid set is 380


In [ ]:
images, landmarks = next(iter(train_loader))
plt.figure(figsize=(600/100, 600/100), dpi=100)
plt.scatter(landmarks[0,:,0], landmarks[0,:,1], c = 'g', s = 10)
plt.imshow(images[0].cpu().numpy().transpose(1,2,0).squeeze(), cmap='gray')
#print(landmarks[0, :])

In [5]:
class Network(nn.Module):
    def __init__(self,num_classes=136):
        super().__init__()
        self.model_name='resnet34'
        self.model=models.resnet34(pretrained=False)
        self.model.conv1=nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=2, bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x=self.model(x)
        return x

In [6]:
import sys

def print_overwrite(step, total_step, loss, operation):
    sys.stdout.write('\r')
    if operation == 'train':
        sys.stdout.write("Train Steps: %d/%d  Loss: %.4f " % (step, total_step, loss))   
    else:
        sys.stdout.write("Valid Steps: %d/%d  Loss: %.4f " % (step, total_step, loss))
        
    sys.stdout.flush()

In [7]:
torch.autograd.set_detect_anomaly(True)
network = Network()
network.cuda()    

criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(network.parameters(), lr=0.0001)

loss_min = np.inf
num_epochs = 100

start_time = time.time()
train_losses = []
valid_losses = []
for epoch in range(1,num_epochs+1):
    epoch_time = time.time()
    loss_train = 0
    loss_valid = 0
    running_loss = 0
    
    network.train()
    for step in range(1,len(train_loader)+1):
    
        images, landmarks = next(iter(train_loader))
        
        images = images.cuda()
        landmarks = landmarks.view(landmarks.size(0),-1).cuda() 
        
        predictions = network(images)
        
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        
        # find the loss for the current step
        loss_train_step = criterion(predictions, landmarks)
        
        # calculate the gradients
        loss_train_step.backward()
        
        # update the parameters
        optimizer.step()
        
        loss_train += loss_train_step.item()
        running_loss = loss_train/step
        
        print_overwrite(step, len(train_loader), running_loss, 'train')
        
    network.eval() 
    with torch.no_grad():
        
        for step in range(1,len(valid_loader)+1):
            
            images, landmarks = next(iter(valid_loader))
        
            images = images.cuda()
            landmarks = landmarks.view(landmarks.size(0),-1).cuda()
        
            predictions = network(images)

            # find the loss for the current step
            loss_valid_step = criterion(predictions, landmarks)

            loss_valid += loss_valid_step.item()
            running_loss = loss_valid/step

            print_overwrite(step, len(valid_loader), running_loss, 'valid')
    
    loss_train /= len(train_loader)
    loss_valid /= len(valid_loader)
    train_losses.insert(epoch, loss_train)
    valid_losses.insert(epoch, loss_valid)
    print('\n--------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Valid Loss: {:.4f}'.format(epoch, loss_train, loss_valid))
    print('--------------------------------------------------')
    
    if loss_valid < loss_min:
        loss_min = loss_valid
        torch.save(network.state_dict(), 'Vacancy_2.pth') # 1 : dataset : 300W & openCVDNN, network : resnet34(3,1,2), preprocess : 50% of openCVDNN
                                                          # 2 : dataset 3219->3816(multi-face images not ommitted)

        print("\nMinimum Validation Loss of {:.4f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
        print('Model Saved\n')
        print("Total Elapsed Time : {} s".format(time.time()-epoch_time))
     
print('Training Complete')
print("Total Elapsed Time : {} s".format(time.time()-start_time))

Valid Steps: 190/190  Loss: 93.5284  
--------------------------------------------------
Epoch: 1  Train Loss: 107.0791  Valid Loss: 93.5284
--------------------------------------------------

Minimum Validation Loss of 93.5284 at epoch 1/100
Model Saved

Total Elapsed Time : 150.58252358436584 s
Valid Steps: 190/190  Loss: 65.5255 
--------------------------------------------------
Epoch: 2  Train Loss: 79.3613  Valid Loss: 65.5255
--------------------------------------------------

Minimum Validation Loss of 65.5255 at epoch 2/100
Model Saved

Total Elapsed Time : 153.7887179851532 s
Valid Steps: 190/190  Loss: 39.7065 
--------------------------------------------------
Epoch: 3  Train Loss: 50.5250  Valid Loss: 39.7065
--------------------------------------------------

Minimum Validation Loss of 39.7065 at epoch 3/100
Model Saved

Total Elapsed Time : 155.32166814804077 s
Valid Steps: 190/190  Loss: 18.7321 
--------------------------------------------------
Epoch: 4  Train Loss: 2

Valid Steps: 190/190  Loss: 3.0039 
--------------------------------------------------
Epoch: 34  Train Loss: 2.9839  Valid Loss: 3.0039
--------------------------------------------------
Valid Steps: 190/190  Loss: 2.7547 
--------------------------------------------------
Epoch: 35  Train Loss: 2.8650  Valid Loss: 2.7547
--------------------------------------------------
Valid Steps: 190/190  Loss: 2.9679 
--------------------------------------------------
Epoch: 36  Train Loss: 2.8408  Valid Loss: 2.9679
--------------------------------------------------
Valid Steps: 190/190  Loss: 2.7520 
--------------------------------------------------
Epoch: 37  Train Loss: 2.7608  Valid Loss: 2.7520
--------------------------------------------------
Valid Steps: 190/190  Loss: 2.4824 
--------------------------------------------------
Epoch: 38  Train Loss: 2.6430  Valid Loss: 2.4824
--------------------------------------------------

Minimum Validation Loss of 2.4824 at epoch 38/100
Model Sav


Minimum Validation Loss of 1.7578 at epoch 72/100
Model Saved

Total Elapsed Time : 153.45125222206116 s
Valid Steps: 190/190  Loss: 1.8308 
--------------------------------------------------
Epoch: 73  Train Loss: 1.7592  Valid Loss: 1.8308
--------------------------------------------------
Valid Steps: 190/190  Loss: 2.4108 
--------------------------------------------------
Epoch: 74  Train Loss: 1.7566  Valid Loss: 2.4108
--------------------------------------------------
Valid Steps: 190/190  Loss: 1.8094 
--------------------------------------------------
Epoch: 75  Train Loss: 1.7498  Valid Loss: 1.8094
--------------------------------------------------
Valid Steps: 190/190  Loss: 1.8241 
--------------------------------------------------
Epoch: 76  Train Loss: 1.7241  Valid Loss: 1.8241
--------------------------------------------------
Valid Steps: 190/190  Loss: 1.8493 
--------------------------------------------------
Epoch: 77  Train Loss: 1.6958  Valid Loss: 1.8493
-----

In [9]:
import csv
with open('Vacancy_2_epochs.csv', 'w', newline='') as f: 
    writer = csv.writer(f) 
    writer.writerow(train_losses) 
    writer.writerow(valid_losses) 